## Loading data into neo4j


In [5]:
# from neo4j import GraphDatabase

# class Neo4jConnect:
#     def __init__(self, user, password):
#         self.driver = GraphDatabase.driver("bolt://localhost:7687", auth=(user, password))
#         self.user = user
#     def close(self):
#         self.driver.close()
        
#     def query(self, query):
#         session = self.driver.session(database="cran")
#         result = session.run(query)
#         return result
    
#     def clean_database(self):
#         """
#         Delete all nodes and edge in Neo4j test container
#         """
#         q = "MATCH (n) DETACH DELETE (n)"
#         try:
#             session = self.driver.session()
#             session.run(q)
#         except Exception as e:
#             print(e)
    
#     def test_neo4j_connection(self):
#         """Test neo4j connection
#         """
#         session = self.driver.session()
#         result = session.run("Match () Return 1 Limit 1")
#         return result
#     def __str__(self):
#         return f'Connection successful for user: {self.user}.\nConnected to Neo4J database uri: {self.uri}'
    
# connection = Neo4jConnect('admin', 'cran2graph')

In [1]:
## Setting up connections

from graphutils.utils import Neo4jConnect

connection = Neo4jConnect('admin', 'cran2graph')

In [2]:
# Testing the connection to neo4j
test_query = """
MATCH (a:Person {name: 'Jing Zhang'})-[:MAINTAINS]->(p:Package)
RETURN a,p
LIMIT 10;
"""

package_maintainer = connection.query(db="neo4j", query=test_query).data()
package_maintainer

TransientError: {code: Memgraph.TransientError.MemgraphError.MemgraphError} {message: Tried to retrieve an unknown database "neo4j".}

In [18]:
# Testing the connection to neo4j
test_query = """
MATCH (a:Person {name: 'Jing Zhang'})-[:MAINTAINS]->(p:Package)
RETURN a,p
LIMIT 10;
"""

package_maintainer = connection.query(db="neo4j", query=test_query).data()
package_maintainer

[{'a': {'name': 'Jing Zhang'},
  'p': {'license': 'GPL-2',
   'latestPublishedDate': '2021-08-09',
   'md5sum': 'ea6fce4d90768de096e57d1b809d15f5',
   'name': 'ggrisk',
   'description': 'The risk plot may be one of the most commonly used figures in \n    tumor genetic data analysis. We can conclude the following two points: \n    Comparing the prediction results of the model with the real survival situation \n    to see whether the survival rate of the high-risk group is lower than that of the \n    low-level group, and whether the survival time of the high-risk group is \n    shorter than that of the low-risk group. The other is to compare the heat \n    map and scatter plot to see the correlation between the predictors and the \n    outcome.',
   'version': '1.3'}},
 {'a': {'name': 'Jing Zhang'},
  'p': {'license': 'GPL-3',
   'latestPublishedDate': '2021-08-03',
   'md5sum': 'f89af897b6b03444fdfa63d91d51ef0a',
   'name': 'do',
   'description': 'Flexibly convert data between long a

: 

### Create constraint on the various nodes

In [4]:
# Create a constraint on 'Package' nodes based on the 'package' property
constraint_package_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (p:Package) REQUIRE p.name IS UNIQUE"

# Create a constraint on 'Person' nodes based on the 'person' property
constraint_person_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (a:Person) REQUIRE a.name IS UNIQUE"

# Create a constraint on 'Person' nodes based on the 'person' property
constraint_license_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (l:License) REQUIRE l.type IS UNIQUE"

# # Create a constraint on 'Person' nodes based on the 'person' property
# constraint_license_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (l:License) REQUIRE l.license IS UNIQUE"

# Create a constraint on 'Person' nodes based on the 'person' property
constraint_institution_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (i:Institution) REQUIRE i.type IS UNIQUE"

# Create a constraint on 'Person' nodes based on the 'person' property
constraint_organization_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (o:Organization) REQUIRE o.name IS UNIQUE"

# Create a constraint on 'Person' nodes based on the 'person' property
constraint_dependency_query = "CREATE CONSTRAINT IF NOT EXISTS FOR (d:Dependency) REQUIRE d.name IS UNIQUE"

# Create an index on the 'person' property of 'Person' nodes
person_index_query = "CREATE INDEX ON :Person(person)"

constraints = [constraint_dependency_query, constraint_organization_query, constraint_license_query, constraint_institution_query, constraint_person_query, constraint_package_query]

# for constraint in constraints:
#     print(f"Creating the constraints with: {constraint}")
#     connection.query(constraint)
# connection.close()


#db_session = establish_connection()

for constraint in constraints:
    print(f"Creating the constraints with: {constraint}")
    connection.query(db="neo4j", query=constraint)
#db_session.close()
    

Creating the constraints with: CREATE CONSTRAINT IF NOT EXISTS FOR (d:Dependency) REQUIRE d.name IS UNIQUE
Creating the constraints with: CREATE CONSTRAINT IF NOT EXISTS FOR (o:Organization) REQUIRE o.name IS UNIQUE
Creating the constraints with: CREATE CONSTRAINT IF NOT EXISTS FOR (l:License) REQUIRE l.type IS UNIQUE
Creating the constraints with: CREATE CONSTRAINT IF NOT EXISTS FOR (i:Institution) REQUIRE i.type IS UNIQUE
Creating the constraints with: CREATE CONSTRAINT IF NOT EXISTS FOR (a:Person) REQUIRE a.name IS UNIQUE
Creating the constraints with: CREATE CONSTRAINT IF NOT EXISTS FOR (p:Package) REQUIRE p.name IS UNIQUE


In [ ]:
# ## Loading package nodes from csv file
# load_package_query = """
# // Load 'Package' nodes from the CSV file
# LOAD CSV WITH HEADERS FROM 'file:///cran_data.csv' AS row
# CALL {
#     WITH row
#     MERGE (p:Package {
#         name: row.package,
#         version: row.version,
#         license: row.license,
#         md5sum: row.md5sum,
#         description: row.description,
#         latestPublishedDate: row.published_date
#     })
#     ON CREATE SET
#         p.name = row.package,
#         p.version = row.version,
#         p.license = row.license,
#         p.md5sum = row.md5sum,
#         p.description = row.description,
#         p.latestPublishedDate = row.published_date
#     ON MATCH SET
#         p.name = row.package,
#         p.version = row.version,
#         p.license = row.license,
#         p.md5sum = row.md5sum,
#         p.description = row.description,
#         p.latestPublishedDate = row.published_date
# } IN TRANSACTIONS OF 10000 ROWS;
# """
# print(f"loading query....")
# connection.query(db="neo4j", query=load_package_query)
# connection.close()
# print(f"end operation....")

In [6]:
## THis fix the issue
## Loading package nodes from csv file
load_package_query = """
// Load 'Package' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_df.csv' AS row
CALL {
    WITH row
    MERGE (p:Package {name: row.package})
    ON CREATE SET
        p.version = row.version,
        p.license = row.license,
        p.md5sum = row.md5sum,
        p.description = row.description,
        p.latestPublishedDate = row.published_date
    ON MATCH SET
        p.version = row.version,
        p.license = row.license,
        p.md5sum = row.md5sum,
        p.description = row.description,
        p.latestPublishedDate = row.published_date
} IN TRANSACTIONS OF 10000 ROWS;
"""
print(f"loading query....")
connection.query(db="neo4j", query=load_package_query)
connection.close()
print(f"end operation....")

loading query....
end operation....


In [7]:
## working comment
## Loading person nodes from csv file
load_person_query = """
// Load 'person' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_df.csv' AS row
CALL {
    WITH row
    MERGE (a:Person {
        name: COALESCE(row.author, "N/A")
    })
} IN TRANSACTIONS OF 10000 ROWS;
"""
print(f"loading query....")
connection.query(db="neo4j", query=load_person_query)
connection.close()
print(f"end operation....")

loading query....
end operation....


In [8]:
## working comment
## Loading imports nodes from csv file
load_import_query = """
// Load 'person' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_df.csv' AS row
CALL {
    WITH row
    MERGE (p:Package {
        name: COALESCE(row.imports, "N/A")
    })
} IN TRANSACTIONS OF 10000 ROWS;
"""
print(f"loading query....")
connection.query(db="neo4j", query=load_import_query)
connection.close()
print(f"end operation....")

loading query....
end operation....


In [9]:
## THis fix the issue
## Loading package nodes from csv file
## Loading person nodes from csv file
load_license_query = """
// Load 'person' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_df.csv' AS row
CALL {
    WITH row
    MERGE (l:License {
        type: COALESCE(row.license, "N/A")
    })
} IN TRANSACTIONS OF 10000 ROWS;
"""
print(f"loading query....")
connection.query(db="neo4j", query=load_license_query)
connection.close()
print(f"end operation....")

loading query....
end operation....


In [10]:
## THis fix the issue
## Loading package nodes from csv file
## Loading person nodes from csv file
load_institution_query = """
// Load 'person' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_df.csv' AS row
CALL {
    WITH row
    MERGE (i:Institution {
        type: COALESCE(row.institution, "N/A")
    })
} IN TRANSACTIONS OF 10000 ROWS;
"""
print(f"loading query....")
connection.query(db="neo4j", query=load_institution_query)
connection.close()
print(f"end operation....")

loading query....
end operation....


In [11]:
## THis fix the issue
## Loading package nodes from csv file
## Loading person nodes from csv file
load_organization_query = """
// Load 'person' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_df.csv' AS row
CALL {
    WITH row
    MERGE (o:Organization {
        name: COALESCE(row.sponsor, "N/A")
    })
} IN TRANSACTIONS OF 10000 ROWS;
"""
print(f"loading query....")
connection.query(db="neo4j", query=load_organization_query)
connection.close()
print(f"end operation....")

loading query....
end operation....


In [12]:
## Loading contributed_to relationships
rel_contributed_to_query = """
// Load 'person' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_df.csv' AS row
MATCH (p:Package { name: row.package })
MATCH (a:Person { name: COALESCE(row.author, "N/A")})

MERGE (a)-[:CONTRIBUTED_TO]->(p);

"""
print(f"loading query....")
connection.query("neo4j", rel_contributed_to_query)
connection.close()
print(f"end operation....")

loading query....
end operation....


In [13]:
## Loading contributed_to relationships
rel_maintains_query = """
// Load 'person' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_df.csv' AS row
MATCH (p:Package { name: row.package })
MATCH (a:Person { name: COALESCE(row.maintainer_name,"N/A")})

MERGE (a)-[:MAINTAINS]->(p);

"""
print(f"loading query....")
connection.query("neo4j", rel_maintains_query)
connection.close()
print(f"end operation....")

loading query....
end operation....


In [14]:
## Loading contributed_to relationships
rel_license_query = """
// Load 'person' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_df.csv' AS row
MATCH (p:Package { name: row.package })
MATCH (l:License { type: COALESCE(row.license,"N/A")})

MERGE (p)-[:LICENSE_BY]-(l);

"""
print(f"loading query....")
connection.query("neo4j",rel_license_query)
connection.close()
print(f"end operation....")

loading query....
end operation....


In [15]:
## Loading contributed_to relationships
rel_organization_query = """
// Load 'person' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_df.csv' AS row
MATCH (p:Package { name: COALESCE(row.package,"N/A")})
MATCH (o:Organization { name: COALESCE(row.sponsor ,"N/A")})

MERGE (p)-[:SUPPORTED_BY]->(o);

"""
print(f"loading query....")
connection.query("neo4j",rel_organization_query)
connection.close()
print(f"end operation....")

loading query....
end operation....


In [16]:
## Loading contributed_to relationships
rel_institution_query = """
// Load 'person' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_df.csv' AS row
MATCH (a:Person { name: COALESCE(row.maintainer_name,"N/A")})
MATCH (i:Institution { type: COALESCE(row.institution ,"N/A")})

MERGE (a)-[:WORKS_AT]->(i);

"""
print(f"loading query....")
connection.query("neo4j",rel_institution_query)
connection.close()
print(f"end operation....")

loading query....
end operation....


In [17]:
## Loading contributed_to relationships
rel_dependency_query = """
// Load 'person' nodes from the CSV file
LOAD CSV WITH HEADERS FROM 'file:///cran_df.csv' AS row
MATCH (p:Package { name: row.package })
MATCH (d:Package { name: COALESCE(row.imports,"N/A")})

MERGE (p)-[:DEPENDS_ON]->(d);

"""
print(f"loading query....")
connection.query("neo4j", rel_dependency_query)
connection.close()
print(f"end operation....")

loading query....
end operation....


In [30]:
import igraph
from neo4j import GraphDatabase

# Define a function to retrieve data from Neo4j
def get_package_data(tx):
    query = "MATCH (p:Package) RETURN p.package, p.description limit 25"
    result = tx.run(query)
    return result.data()

# Connect to your Neo4j database
uri = "neo4j://localhost:7687"
username = "admin"
password = "cran2graph"

# Create a Neo4j driver
driver = GraphDatabase.driver(uri, auth=(username, password))


# Retrieve package data from Neo4j
with driver.session(database="cran") as session:
    data = session.read_transaction(get_package_data)

# Preprocess the input text (e.g., remove stopwords and tokenize)
input_text = "package to perform data transformation and cleaning"

# Create a list of package descriptions
package_descriptions = [package["description"] for package in data]

C:\Users\Dee\AppData\Local\Temp\ipykernel_92308\1262747138.py:21: DeprecationWarning: read_transaction has been renamed to execute_read
  data = session.read_transaction(get_package_data)


[{'p.package': 'A3', 'p.description': 'Supplies tools for tabulating and analyzing the results of predictive models. The methods employed are applicable to virtually any predictive model and make comparisons between different methodologies straightforward.'}, {'p.package': 'AalenJohansen', 'p.description': 'Provides the conditional Nelson-Aalen and Aalen-Johansen estimators. The methods are based on Bladt & Furrer (2023), in preparation.'}, {'p.package': 'AATtools', 'p.description': 'Compute approach bias scores using different scoring algorithms,\n    compute bootstrapped and exact split-half reliability estimates,\n    and compute confidence intervals for individual participant scores.'}, {'p.package': 'ABACUS', 'p.description': 'A set of Shiny apps for effective communication and understanding in statistics. The current version includes properties of normal distribution, properties of sampling distribution, one-sample z and t tests, two samples independent (unpaired) t test and anal